# Warm-Up Task: Wine quality prediction with PyTorch
This was a side task to get back into the PyTorch framework for deep learning before diving into the main task—creating and training an ECG neural network to determine diseases from ECG signals.

## Libraries
Well, for this task, all we need is PyTorch and NumPy. We could use scikit-learn for an efficient train-test split, but since this is a simple dataset and just a warm-up, it's better to keep things minimal.

In [2]:
import torch
import torch.utils.data as utils
from torch import nn
import numpy as np

torch.__version__

'2.6.0+cu126'

## Model
The model could have been simpler, but there was a bit of experimentation with dropout and ReLU to see how much of a difference they would make. Otherwise, it’s just linear layers, which don’t need much explanation.

In [3]:
class WineModel(nn.Module):
    def __init__(self, dtype=torch.float32):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(in_features=11, out_features=256, dtype=dtype),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(in_features=256, out_features=256, dtype=dtype),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(in_features=256, out_features=256, dtype=dtype),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(in_features=256, out_features=256, dtype=dtype),
            nn.ReLU(True),
            nn.Dropout(0.2),
            nn.Linear(in_features=256, out_features=1, dtype=dtype)
        )

    def forward(self, x):
        return self.features(x)

## Processing Data
A simple CSV load with NumPy, splitting it into actual data (x) and labels (y). After that, I split them in the traditional 80:20 (train:test) ratio, no validation needed here. Finally, I passed them through a dataset and DataLoader.

In [4]:
data = np.loadtxt("./data/winequality-white.csv", delimiter=';', skiprows=1)

data_x = data[:, 0:11]
data_y = data[:, 11]

train_split = int(0.8 * len(data_x))

X_train, y_train = data_x[:train_split], data_y[:train_split]
X_test, y_test = data_x[train_split:], data_y[train_split:]

tensor_x_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)

tensor_x_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)

dataset_train = utils.TensorDataset(tensor_x_train,tensor_y_train)
dataloader_train = utils.DataLoader(dataset_train)

dataset_test = utils.TensorDataset(tensor_x_test,tensor_y_test)
dataloader_test = utils.DataLoader(dataset_test)

## Device Agnostics, Loss Function, and Optimizer
Since the model is built with linear layers, the choice of loss function is a no-brainer, L1Loss. As for the optimizer, I just went with one I like, no special reason.

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = WineModel().to(device)

loss_fn = nn.L1Loss()

optimizer = torch.optim.Adam(model.parameters(), 0.001)

## Training & Testing Loop
Well, this one I remembered very well, thanks to this song: https://www.youtube.com/watch?v=Nutpusq_AFw. Thank you, David!

In [8]:
epochs = 50
for epoch in range(epochs):
    train_loss = 0
    test_loss = 0

    # Set the model to train mode
    model.train()
    for x, y in dataloader_train:
        x = x.to(device)
        y = y.to(device)

        # Forward pass
        pred = model(x)

        # Calculate the loss
        loss = loss_fn(pred.flatten(), y)

        # Zero the optimizer gradients
        optimizer.zero_grad()

        # Perform backpropagation
        loss.backward()

        # Gradient descent
        optimizer.step()

        # Accumulate train loss
        train_loss += loss.item()

    # Set the model to evaluation mode
    model.eval()

    # Entering torch inference mode which is torch.no_grad() on steroids - both disable gradient tracking
    with torch.inference_mode():
        for x, y in dataloader_test:
            x = x.to(device)
            y = y.to(device)

            # Forward pass
            pred = model(x)

            # Calculate the loss
            loss = loss_fn(pred.flatten(), y)

            # Accumulate test loss
            test_loss += loss.item()

    # Print what is happenning
    print(f"Epoch {epoch + 1}/{epochs} | "
          f"Train loss: {train_loss / len(dataloader_train):.4f}, "
          f"Test loss: {test_loss / len(dataloader_test):.4f}")

Epoch 1/50 | Train loss: 0.7290, Test loss: 0.5310
Epoch 2/50 | Train loss: 0.7262, Test loss: 0.5310
Epoch 3/50 | Train loss: 0.7326, Test loss: 0.5691
Epoch 4/50 | Train loss: 0.7279, Test loss: 0.5452
Epoch 5/50 | Train loss: 0.7260, Test loss: 0.5399
Epoch 6/50 | Train loss: 0.7199, Test loss: 0.5479
Epoch 7/50 | Train loss: 0.7090, Test loss: 0.5949
Epoch 8/50 | Train loss: 0.7117, Test loss: 0.5421
Epoch 9/50 | Train loss: 0.7107, Test loss: 0.4940
Epoch 10/50 | Train loss: 0.7144, Test loss: 0.5420
Epoch 11/50 | Train loss: 0.7162, Test loss: 0.5723
Epoch 12/50 | Train loss: 0.7155, Test loss: 0.5409
Epoch 13/50 | Train loss: 0.7184, Test loss: 0.5377
Epoch 14/50 | Train loss: 0.6929, Test loss: 0.5430
Epoch 15/50 | Train loss: 0.6956, Test loss: 0.5531
Epoch 16/50 | Train loss: 0.6984, Test loss: 0.5312
Epoch 17/50 | Train loss: 0.6970, Test loss: 0.5939
Epoch 18/50 | Train loss: 0.7001, Test loss: 0.5676
Epoch 19/50 | Train loss: 0.6923, Test loss: 0.5113
Epoch 20/50 | Train l

## Save the model's state dictionary

In [9]:
torch.save(model.state_dict(), "./models/winemodel.pth")

## Show predictions

In [10]:
def print_pred(vals, ans):
    model.eval()
    with torch.inference_mode():
        pred = model(vals)
        print(f"Prediction: {pred}, Answer: {ans}")

In [12]:
model = WineModel().to(device)
model.load_state_dict(torch.load("./models/winemodel.pth", weights_only=True))

print_pred(torch.tensor(data[0, :11], dtype=torch.float32).to(device), data[0][11])
print_pred(torch.tensor(data[1, :11], dtype=torch.float32).to(device), data[1][11])
print_pred(torch.tensor(data[2, :11], dtype=torch.float32).to(device), data[2][11])

Prediction: tensor([5.9243], device='cuda:0'), Answer: 6.0
Prediction: tensor([5.5710], device='cuda:0'), Answer: 6.0
Prediction: tensor([5.9394], device='cuda:0'), Answer: 6.0
